In [ ]:
import pandas as pd
import json
import requests
import datetime
from config import postgres_url_user_pass
from config import postgres_server

In [ ]:
#URL found from viewing web inspector in web browser to find the current list of issues
url = "https://seeclickfix.com/api/v2/issues?min_lat=39.03211937423035&min_lng=-95.71889877319336&max_lat=39.052118348329074&max_lng=-95.6312656402588&status=open%2Cacknowledged%2Cclosed&fields%5Bissue%5D=id%2Csummary%2Cdescription%2Cstatus%2Clat%2Clng%2Caddress%2Cmedia%2Ccreated_at%2Cacknowledged_at%2Cclosed_at&page=1"

In [ ]:
#grab an initial response to grab the total number of pages
response = requests.get(url).json()
total_pages = response['metadata']['pagination']['pages']

In [ ]:
#define a function to get urls from the initial URL 
def get_issues(url):
    #create a temp list for all issues to return
    all_issues = []
    for i in range(1,total_pages +1):
        print(f"getting page {i} of {total_pages}")
        response = requests.get(url).json()
        next_page_url = response['metadata']['pagination']['next_page_url']
        page_issues = response['issues']
        #add current list of issues to the existing list of issues
        all_issues.extend(page_issues)
        #set URL to next page to get the next set of issues
        url = next_page_url
    #return the list of all issues
    return all_issues

In [ ]:
# call function to get all issues
all_issues = get_issues(url)

In [ ]:
#create dataframe from issue list
issues_df = pd.DataFrame(all_issues)
#set index to the issues id
issues_df = issues_df.set_index('id')

In [ ]:
def clean_summary(df):
    #convert dates
    df['created_at'] = pd.to_datetime(
        df['created_at'], format="%Y/%m/%d", utc=True)
    df['acknowledged_at'] = pd.to_datetime(
        df['acknowledged_at'], format="%Y/%m/%d", utc=True)
    df['closed_at'] = pd.to_datetime(
        df['closed_at'], format="%Y/%m/%d", utc=True)
    # calculate time to acknowledge
    df['time to acknowledge'] = df['acknowledged_at'] - df['created_at']
    df['time to close'] = df['closed_at'] - df['created_at']
    
    #return daframe and replace known errant values
    return df.replace({"Curb and Gutter": "Curb and Gutter Repair",
                       "Junk Yard?": "Property Maintenance & Code Violations",
                       "skatepark light": "safety lighting",
                       "Dead animal on sidewalk ": "Police Animal Control",
                       "Loud pedestrian crossing ": "Street Sign Repair",
                       "Pothole": "Street Repair"})


In [ ]:
# call function to clean the summary column
issues_df = clean_summary(issues_df)

In [ ]:
status_list = issues_df['status'].value_counts()

In [ ]:
summary_list = issues_df['summary'].value_counts()
summary_list

In [ ]:
summary_df = pd.DataFrame(issues_df.groupby('summary')['status'].count())

In [ ]:
summary_df.plot(kind="bar")

In [ ]:
status = pd.DataFrame(issues_df.groupby('status')['status'].count())
status.plot(kind="bar")

In [ ]:
issues_df['time to acknowledge']

In [ ]:
issues_df.groupby('time to acknowledge')['time to acknowledge'].count()

In [ ]:
#save to CSV, in case you want to
issues_df.to_csv("all_issues.csv")

# Let's rock this out to the DB and go from there

In [ ]:
from sqlalchemy import create_engine
import psycopg2
engine = create_engine(
    f'postgresql+psycopg2://{postgres_url_user_pass}@{postgres_server}/seeClickFix', echo=False)
issues_df.to_sql('seeClickFixExport',engine)

